## Imports

In [139]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from os import listdir
from os.path import isfile, join
from PIL import Image
from torchvision import datasets, models
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import time as time
import copy
from torchvision.datasets import OxfordIIITPet
from torch.optim import lr_scheduler

from tqdm import tqdm

## Fields

In [140]:



num_classes = 37 #Number of classes in dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
image_size = 224
batch_size = 128

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

path = ""

## Methods

In [141]:
def set_parameter_requires_grad(model, feature_extracting):
    params_to_update = list(model.named_parameters())
    
    if feature_extracting:
        for name, param in params_to_update:
            param.requires_grad = False      
    else:
        for module, param in zip(model.modules(), model.parameters()):
            if isinstance(module, nn.BatchNorm2d):
                param.requires_grad = False
            else:
                param.requires_grad = True
            



def initialize_model(model_name, num_classes, feature_extracting): #Initialize Resnet

    if model_name == "resnet":

        model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True) 
        
        set_parameter_requires_grad(model, feature_extracting)
        
        num_ftrs = model.fc.in_features
        head = nn.Sequential(nn.BatchNorm1d(num_ftrs),
                             nn.ReLU(),
                             nn.Dropout(0.25),
                nn.Linear(num_ftrs, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(),
                nn.Dropout(0.25),
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(256, num_classes))
        
        model.fc = head#nn.Linear(num_ftrs,num_classes) # Update last layer to binary classification (Dog/cat)
        input_size = 224 #"Finally, notice that inception_v3 requires the input size to be (299,299), whereas all of the other models expect (224,224)."
        model = model.to(device)
        
    return model, input_size



def readData(target_type = 'breeds'):
    breed_to_species = {0:0, 1:1, 2:1, 3:1, 4:1, 5:0, 6:0, 7:0, 8:1, 9:0, 10:1,
                    11:0, 12:1, 13:1, 14:1, 15:1, 16:1, 17:1, 18:1, 19:1, 20:0,
                    21:1, 22:1, 23:0, 24:1, 25:1, 26:0, 27:0, 28:1, 29:1, 30:1,
                    31:1, 32:0, 33:0, 34:1, 35:1, 36:1
                   }
    
    transform_train = T.Compose([
                           T.Resize(256),
                           T.CenterCrop(image_size),
                           T.RandomHorizontalFlip(p=0.5),
                           #T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
                           #T.RandomHorizontalFlip(),
                           T.RandomRotation(degrees=90),
                           T.ToTensor(),
                           T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                           #T.RandomResizedCrop(size=image_size)
                          ])
    

    
    transform_test = T.Compose([T.Resize(256),
                            T.CenterCrop(image_size),
                           T.ToTensor(),
                           T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                          ])
    if target_type == 'species':
        target_function = (lambda x: breed_to_species.get(x))
    elif target_type == 'breeds':
        target_function = None
    else:
        raise Exception("Wrong target type")
        
    trainval = OxfordIIITPet(path,
                             target_types= "category",
                             transform=transform_train,
                             split = 'trainval',
                             target_transform = target_function
                            )
    test = OxfordIIITPet(path,
                         target_types= "category",
                         transform=transform_test,
                         split = 'test',
                         target_transform = target_function
                        )
                         
    #train, val = splitData(trainval, 0.95, 0.05)
    return trainval, test #train, val, test
    
def splitData(dataset,nrTrain,nrVal):
    len1 = int(nrTrain*len(dataset))
    len2 = int(len(dataset) - len1)
    train, val = torch.utils.data.random_split(dataset, [len1,len2])
    return train, val

def show_example(img, label):
    print('Label: ', train_dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))




In [142]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()

                # statistics
                running_loss += float(loss.item() * inputs.size(0))
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                pass#scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    #model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [143]:
def score(model, dataloader, criterion):
    running_corrects = 0

    with torch.no_grad():
        model.eval()
        for inputs, labels in tqdm(dataloader):
            
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            running_corrects += torch.sum(preds == labels.data)

    acc = running_corrects.double() / len(dataloader.dataset)
    print('Acc: {:4f}'.format(acc))

## Model evaluation

In [144]:
train, val    = readData(
                            #target_type = 'species'
                            target_type = 'breeds'
                            )

In [145]:
#train, _ = splitData(train, 0.20, 0.8)
#val, _ = splitData(val, 0.20,0.8)

In [146]:
model,input_size = initialize_model(model_name = "resnet",
                                    num_classes = 37,
                                    feature_extracting = True)

Using cache found in C:\Users\GTSA - Infinity/.cache\torch\hub\pytorch_vision_v0.10.0


In [147]:
dataloaders = {label: torch.utils.data.DataLoader(data, batch_size=batch_size,
                                         shuffle=True, num_workers=0)
          for label, data in zip(['train', 'val'],[train, val])}

test_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size,
                                         shuffle=False, num_workers=0)
epochs = 10
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(),
                       lr=1e-7, 
                       weight_decay=1e-5,
                       betas=(0.9, 0.999),
                      )



#scheduler = lr_scheduler.StepLR(optimizer, step_size=3,gamma=0.1)



scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                max_lr=1e-3,
                                                steps_per_epoch=len(dataloaders['train']), epochs=epochs)


best_model, val_acc_history = train_model(model = model,
                                          num_epochs = epochs,
                                          dataloaders = dataloaders,
                                          criterion = criterion,
                                          optimizer = optimizer,
                                          scheduler = scheduler
                                        )

Epoch 0/9
----------
train Loss: 3.6924 Acc: 0.0470
val Loss: 3.3888 Acc: 0.1962
Epoch 1/9
----------
train Loss: 3.0554 Acc: 0.2440
val Loss: 2.1955 Acc: 0.7531
Epoch 2/9
----------
train Loss: 2.0294 Acc: 0.5601
val Loss: 1.1617 Acc: 0.8357
Epoch 3/9
----------
train Loss: 1.3936 Acc: 0.6609
val Loss: 0.7807 Acc: 0.8621
Epoch 4/9
----------
train Loss: 1.0922 Acc: 0.7228
val Loss: 0.6385 Acc: 0.8629
Epoch 5/9
----------
train Loss: 0.9573 Acc: 0.7481
val Loss: 0.5609 Acc: 0.8675
Epoch 6/9
----------
train Loss: 0.8914 Acc: 0.7543
val Loss: 0.5258 Acc: 0.8738
Epoch 7/9
----------
train Loss: 0.8629 Acc: 0.7579
val Loss: 0.5151 Acc: 0.8697
Epoch 8/9
----------
train Loss: 0.8354 Acc: 0.7685
val Loss: 0.5090 Acc: 0.8716
Epoch 9/9
----------
train Loss: 0.8224 Acc: 0.7753
val Loss: 0.5011 Acc: 0.8735
Training complete in 7m 43s
Best val Acc: 0.873808


In [148]:
score(best_model, test_dataloader, criterion)

100%|██████████| 29/29 [00:20<00:00,  1.43it/s]

Acc: 0.873535


In [149]:
torch.save(model.state_dict(), 'tensor')

In [162]:
layer_names = []
for idx, (name, param) in enumerate(model.named_parameters()):
    layer_names.append(name)
    print(f'{idx}: {name}')

layer_names.reverse()

# learning rate
lr      = 1e-2
lr_mult = 0.9

# placeholder
parameters = []

# store params & learning rates
for idx, name in enumerate(layer_names):
    
    # display info
    print(f'{idx}: lr = {lr:.6f}, {name}')
    
    # append layer parameters
    parameters += [{'params': [p for n, p in model.named_parameters() if n == name and p.requires_grad],
                    'lr':     lr}]
    
    # update learning rate
    lr *= lr_mult

optimizer = optim.Adam(parameters)

0: conv1.weight
1: bn1.weight
2: bn1.bias
3: layer1.0.conv1.weight
4: layer1.0.bn1.weight
5: layer1.0.bn1.bias
6: layer1.0.conv2.weight
7: layer1.0.bn2.weight
8: layer1.0.bn2.bias
9: layer1.1.conv1.weight
10: layer1.1.bn1.weight
11: layer1.1.bn1.bias
12: layer1.1.conv2.weight
13: layer1.1.bn2.weight
14: layer1.1.bn2.bias
15: layer2.0.conv1.weight
16: layer2.0.bn1.weight
17: layer2.0.bn1.bias
18: layer2.0.conv2.weight
19: layer2.0.bn2.weight
20: layer2.0.bn2.bias
21: layer2.0.downsample.0.weight
22: layer2.0.downsample.1.weight
23: layer2.0.downsample.1.bias
24: layer2.1.conv1.weight
25: layer2.1.bn1.weight
26: layer2.1.bn1.bias
27: layer2.1.conv2.weight
28: layer2.1.bn2.weight
29: layer2.1.bn2.bias
30: layer3.0.conv1.weight
31: layer3.0.bn1.weight
32: layer3.0.bn1.bias
33: layer3.0.conv2.weight
34: layer3.0.bn2.weight
35: layer3.0.bn2.bias
36: layer3.0.downsample.0.weight
37: layer3.0.downsample.1.weight
38: layer3.0.downsample.1.bias
39: layer3.1.conv1.weight
40: layer3.1.bn1.weight
4

In [150]:
set_parameter_requires_grad(best_model, False)

    
"""optimizer = optim.Adam(model.parameters(),
                       lr=1e-6, 
                       weight_decay=1e-5,
                       betas=(0.9, 0.999),
                      )"""

finetune_epochs = 15
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                max_lr=1e-4,
                                                steps_per_epoch=len(dataloaders['train']),
                                                epochs=finetune_epochs)

best_model, val_acc_history = train_model(model = best_model,
                                          num_epochs = finetune_epochs,
                                          dataloaders = dataloaders,
                                          criterion = criterion,
                                          optimizer = optimizer,
                                          scheduler = scheduler
                                        )

Epoch 0/14
----------
train Loss: 0.7721 Acc: 0.7880
val Loss: 0.4679 Acc: 0.8809
Epoch 1/14
----------
train Loss: 0.7010 Acc: 0.8052
val Loss: 0.4081 Acc: 0.8934
Epoch 2/14
----------
train Loss: 0.5997 Acc: 0.8359
val Loss: 0.4084 Acc: 0.8907
Epoch 3/14
----------
train Loss: 0.4872 Acc: 0.8709
val Loss: 0.3708 Acc: 0.9008
Epoch 4/14
----------
train Loss: 0.4341 Acc: 0.8872
val Loss: 0.3694 Acc: 0.8994
Epoch 5/14
----------
train Loss: 0.3688 Acc: 0.9098
val Loss: 0.3400 Acc: 0.9035
Epoch 6/14
----------
train Loss: 0.3269 Acc: 0.9217
val Loss: 0.3371 Acc: 0.9038
Epoch 7/14
----------
train Loss: 0.2726 Acc: 0.9421
val Loss: 0.3335 Acc: 0.9030
Epoch 8/14
----------
train Loss: 0.2581 Acc: 0.9427
val Loss: 0.3300 Acc: 0.9019
Epoch 9/14
----------
train Loss: 0.2201 Acc: 0.9560
val Loss: 0.3006 Acc: 0.9106
Epoch 10/14
----------
train Loss: 0.1975 Acc: 0.9647
val Loss: 0.3115 Acc: 0.9073
Epoch 11/14
----------
train Loss: 0.1808 Acc: 0.9677
val Loss: 0.2934 Acc: 0.9131
Epoch 12/14
--

In [151]:
score(best_model, test_dataloader, criterion)

100%|██████████| 29/29 [00:20<00:00,  1.44it/s]

Acc: 0.914418


[('weight', Parameter containing:
tensor([[[[ 6.1361e-03, -6.5649e-03,  8.3418e-03,  ...,  4.9540e-02,
            3.1104e-02,  2.5868e-02],
          [ 4.1682e-02,  3.1591e-02,  3.2319e-02,  ...,  3.3264e-02,
            2.9824e-02,  4.1920e-02],
          [ 5.9094e-03, -3.1138e-02, -6.0792e-02,  ..., -9.7410e-02,
           -1.1579e-01, -1.2144e-01],
          ...,
          [-1.1544e-02, -2.4333e-02, -8.8229e-03,  ...,  1.7413e-02,
            2.7027e-03,  1.6626e-02],
          [ 4.6376e-03,  4.8749e-03,  3.6821e-02,  ...,  1.0416e-01,
            7.4349e-02,  5.8992e-02],
          [ 1.7455e-02,  9.3215e-03,  3.1775e-02,  ...,  9.7256e-02,
            8.3895e-02,  9.7059e-02]],

         [[-7.2190e-03, -8.6658e-03,  1.4246e-02,  ...,  3.4098e-02,
            2.5813e-02,  2.4507e-02],
          [ 5.4462e-02,  4.4711e-02,  3.3961e-02,  ...,  1.3182e-02,
            1.9049e-02,  3.8006e-02],
          [ 1.9030e-03, -5.4132e-02, -1.0213e-01,  ..., -1.9261e-01,
           -2.0002e-01, 

0: conv1.weight
1: bn1.weight
2: bn1.bias
3: layer1.0.conv1.weight
4: layer1.0.bn1.weight
5: layer1.0.bn1.bias
6: layer1.0.conv2.weight
7: layer1.0.bn2.weight
8: layer1.0.bn2.bias
9: layer1.1.conv1.weight
10: layer1.1.bn1.weight
11: layer1.1.bn1.bias
12: layer1.1.conv2.weight
13: layer1.1.bn2.weight
14: layer1.1.bn2.bias
15: layer2.0.conv1.weight
16: layer2.0.bn1.weight
17: layer2.0.bn1.bias
18: layer2.0.conv2.weight
19: layer2.0.bn2.weight
20: layer2.0.bn2.bias
21: layer2.0.downsample.0.weight
22: layer2.0.downsample.1.weight
23: layer2.0.downsample.1.bias
24: layer2.1.conv1.weight
25: layer2.1.bn1.weight
26: layer2.1.bn1.bias
27: layer2.1.conv2.weight
28: layer2.1.bn2.weight
29: layer2.1.bn2.bias
30: layer3.0.conv1.weight
31: layer3.0.bn1.weight
32: layer3.0.bn1.bias
33: layer3.0.conv2.weight
34: layer3.0.bn2.weight
35: layer3.0.bn2.bias
36: layer3.0.downsample.0.weight
37: layer3.0.downsample.1.weight
38: layer3.0.downsample.1.bias
39: layer3.1.conv1.weight
40: layer3.1.bn1.weight
4

0: conv1.weight
1: bn1.weight
2: bn1.bias
3: layer1.0.conv1.weight
4: layer1.0.bn1.weight
5: layer1.0.bn1.bias
6: layer1.0.conv2.weight
7: layer1.0.bn2.weight
8: layer1.0.bn2.bias
9: layer1.1.conv1.weight
10: layer1.1.bn1.weight
11: layer1.1.bn1.bias
12: layer1.1.conv2.weight
13: layer1.1.bn2.weight
14: layer1.1.bn2.bias
15: layer2.0.conv1.weight
16: layer2.0.bn1.weight
17: layer2.0.bn1.bias
18: layer2.0.conv2.weight
19: layer2.0.bn2.weight
20: layer2.0.bn2.bias
21: layer2.0.downsample.0.weight
22: layer2.0.downsample.1.weight
23: layer2.0.downsample.1.bias
24: layer2.1.conv1.weight
25: layer2.1.bn1.weight
26: layer2.1.bn1.bias
27: layer2.1.conv2.weight
28: layer2.1.bn2.weight
29: layer2.1.bn2.bias
30: layer3.0.conv1.weight
31: layer3.0.bn1.weight
32: layer3.0.bn1.bias
33: layer3.0.conv2.weight
34: layer3.0.bn2.weight
35: layer3.0.bn2.bias
36: layer3.0.downsample.0.weight
37: layer3.0.downsample.1.weight
38: layer3.0.downsample.1.bias
39: layer3.1.conv1.weight
40: layer3.1.bn1.weight
4

Using cache found in C:\Users\GTSA - Infinity/.cache\torch\hub\pytorch_vision_v0.10.0
